In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, recall_score
from sklearn.inspection import permutation_importance

# EDA


In [ ]:
df = pd.read_csv("ACME-HappinessSurvey2020.csv")

Let's do some exploratory data analysis


In [ ]:
df.info()

There are no missing data but the dataset is small. We should be careful with overfitting.

Let's check the class balance.


In [ ]:
df["Y"].value_counts(normalize=True)

55% are happy and 45% are unhappy. Seems pretty well balanced but this is misleading. More on that later.

For now let's see the summary statistics


In [ ]:
df.describe()

We see that

- X1 has the largest mean (~4.33) and the lowest standard deviation (0.8), which means that satisfaction is high and variability is low. i.e. Most people are happy with little disagreement. X6 shows a similar case with a mean of ~ 4.25 and a std of ~0.81. Likewise for X4 with mean ~ 3.75 and std ~ 0.88.
- X2 has the lowest mean (~2.53) with one of the highest std (~1.11) and a median of 3.0. Since median > mean, the distribution is left skewed meaning there are some very low scores (1s and 2s) that pull the mean below the median. Most reponses cluster around 3 but overall satisfaction is low.
- X5 is also left skewed but the mean is higher (~3.65) with the highest std (~ 1.15). Most people are happy but there seems to be a large division between happy and unhappy people here.

Let's check the distribution of the features by plotting them on bar plots


In [ ]:
feature_explanations = {
    "X1": "X1: Order delivered on time",
    "X2": "X2: Order contents as expected",
    "X3": "X3: Ordered everything wanted",
    "X4": "X4: Paid a good price for order",
    "X5": "X5: Satisfied with courier",
    "X6": "X6: App makes ordering easy",
}

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

for i, col in enumerate(["X1", "X2", "X3", "X4", "X5", "X6"]):
    sns.countplot(data=df, x=col, hue="Y", ax=axes[i])
    axes[i].set_title(f"Distribution of {col} by Happiness")
    axes[i].set_xlabel(feature_explanations[col])
    axes[i].legend(title="Y (Happy)", loc="upper left")

plt.tight_layout()

As discussed earlier,

- Most people were happy about the app being easy to use (X6), paying a good price for their orders (X4) and orders being delivered on time (X1).
- Most people were unhappy because the content of their orders were not as expected (X2).

We also see that

- For X5, happy customers mostly rated their courier highly but the unhappy customers were more divided, explaining the high standard deviation from earlier.
- X3 is a bit of a mixed bag. Most people had a score of 3. If people were able to find the item that they wanted, they would be happy. Otherwise, they would be unhappy.


Let's validate this by looking at which features happy and unhappy customers differ the most as well as the correlation matrix


In [ ]:
df.groupby("Y").mean().T.assign(
    diff=lambda x: x[1] - x[0], corr_with_Y=df.corr()["Y"].drop("Y")
).sort_values("diff", ascending=False)

The biggest variance/gaps are with X1 (~0.45) and X5 (~0.52).

For X1, the mean scores for the unhappy customers are still relatively high whereas for X5, the unhappy customers are leaning towards middle-low.

These two features are also the ones which correlate highest with the target. That is, they are the most likely to predict happiness.

On the opposite end, X2 has the highest negative correlation with Y (but still ~0). Everyone rated it poorly, which means that fixing X2 might improve overall satisfaction but not necessarily convert unhappy customer to happy customers.


In [ ]:
feature_corr = df.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(feature_corr, annot=True, cmap="coolwarm", center=0, fmt=".2f")
plt.title("Feature Correlation Matrix")
plt.tight_layout()


The correlation matrix shows moderate correlations between features, especially X5 and X1 (0.43), X6 and X1 (0.41), and X5 and X3 (0.36). This suggests that some features share some overlapping information.


# Unhappy customers


Recall that 55% of the customers were happy and 45% of the customers were unhappy. The dataset seems well balanced.

However, even if the dataset might be balanced in general, in a business persepective, it is unbalanced.

45% of customers being unhappy is alarming and the company gains more from saving an unhappy customer.

<br>

From our analysis above, we already have a strong candidate: X2, but do we have any other features that can help us predict the truly unhappy customers?


In [ ]:
unhappy_df = df[df["Y"] == 0]
happy_df = df[df["Y"] == 1]

In [ ]:
unhappy_df.apply(lambda x: (x <= 2).sum()).sort_values(ascending=False)

Unhappy customers were most unhappy that their orders were not as expected (X2), that there was something wrong with their courier/delivery (X5) and that some items were out of stock (X3).

<br>

Now that we've identified what unhappy customers complain about most, let's find out which of these features actually predict unhappiness.

Instead of maximizing accuracy, we should maximize **recall** for class 0.

<br>

**Note**: Models predict customers as happy or unhappy. If the model predicts everyone as happy we would still achieve 55% accuracy but that would be useless to us since it did not find any unhappy customers. A model that predicts everyone as unhappy would achieve 45% accuracy but then it would not flag any happy customers.

Recall ensures we prioritize identifying truly unhappy customers.


# Modeling


### Split Dataset and model recall for class 0


We split the dataset into features and target so the model can learn: given a customer's response, predict whether they're happy or unhappy.


In [ ]:
X = df.drop("Y", axis=1)
y = df["Y"]

Instead of a traditional train/test split, we use cross validation since we have a very small dataset and want to avoid overfitting.

We compare recall using the following models:

- Logistic Regression since it's a good linear baseline
- Random Forest. Tree based ensemble, handles non-linear relationships
- Gradient Boosting. Tree based ensemble
- Support Vector Machine (SVM). Finds optimal decision boundaries, works well on small datasets


In [ ]:
y_pred_log_reg = cross_val_predict(LogisticRegression(), X, y)
y_pred_rf = cross_val_predict(RandomForestClassifier(), X, y)
y_pred_gb = cross_val_predict(GradientBoostingClassifier(), X, y)
y_pred_svc = cross_val_predict(SVC(), X, y)  # SVC is SVM but for classification

In [ ]:
print(
    classification_report(y, y_pred_log_reg, target_names=["Unhappy", "Happy"])
    + " Logistic Regression"
    + "\n"
    + classification_report(y, y_pred_rf, target_names=["Unhappy", "Happy"])
    + " Random Forest"
    + "\n"
    + classification_report(y, y_pred_gb, target_names=["Unhappy", "Happy"])
    + " Gradient Boosting"
    + "\n"
    + classification_report(y, y_pred_svc, target_names=["Unhappy", "Happy"])
    + " Support Vector Classifier"
)

For all models (class 0), the results are quite bad. Recall is below 50%.

However, this is our baseline. We might be able to improve by doing a few things like hyperparameter tuning.


### Hyperparameter Tuning


By default, models treat both classes equally. This means that with 55% happy and 45% unhappy, the model might lean towards predicting 'happy' more often.

We can try using class_weight='balanced' to adjust weights inversely proportional to class frequencies. That is, misclassifying an unhappy customer costs more than misclassifying a happy one.


In [ ]:
y_pred_log_reg = cross_val_predict(LogisticRegression(class_weight="balanced"), X, y)
y_pred_rf = cross_val_predict(RandomForestClassifier(class_weight="balanced"), X, y)
y_pred_gb = cross_val_predict(GradientBoostingClassifier(), X, y)
y_pred_svc = cross_val_predict(SVC(class_weight="balanced"), X, y)

In [ ]:
print(
    classification_report(y, y_pred_log_reg, target_names=["Unhappy", "Happy"])
    + " Logistic Regression, balanced"
    + "\n"
    + classification_report(y, y_pred_rf, target_names=["Unhappy", "Happy"])
    + " Random Forest, balanced"
    + "\n"
    + classification_report(y, y_pred_gb, target_names=["Unhappy", "Happy"])
    + " Gradient Boosting"
    + "\n"
    + classification_report(y, y_pred_svc, target_names=["Unhappy", "Happy"])
    + " Support Vector Classifier, balanced"
)

Results are better this time with SVM having the best recall for class 0 at 0.61 from 0.37. But let's see if we can do even better.

What this means in practice is that we went from catching 21 out of 57 unhappy customers to catching 35 out of 57 customers.

<br>

Since SVM scale sensitive (uses distance to find decision boundaries), maybe we can try transforming the data with the standard scaler (removes mean and scaling to unit variance). i.e. mean=0, std=1.


In [ ]:
svc_pipe = Pipeline(
    [("scaler", StandardScaler()), ("svc", SVC(class_weight="balanced"))]
)

In [ ]:
y_pred_svc_pipe = cross_val_predict(svc_pipe, X, y)
y_pred_log_reg = cross_val_predict(LogisticRegression(class_weight="balanced"), X, y)
y_pred_rf = cross_val_predict(RandomForestClassifier(class_weight="balanced"), X, y)
y_pred_gb = cross_val_predict(GradientBoostingClassifier(), X, y)
y_pred_svc = cross_val_predict(SVC(class_weight="balanced"), X, y)

In [ ]:
print(
    classification_report(y, y_pred_log_reg, target_names=["Unhappy", "Happy"])
    + " Logistic Regression, balanced"
    + "\n"
    + classification_report(y, y_pred_rf, target_names=["Unhappy", "Happy"])
    + " Random Forest, balanced"
    + "\n"
    + classification_report(y, y_pred_gb, target_names=["Unhappy", "Happy"])
    + " Gradient Boosting"
    + "\n"
    + classification_report(y, y_pred_svc, target_names=["Unhappy", "Happy"])
    + " Support Vector Classifier, balanced"
    + "\n"
    + classification_report(y, y_pred_svc_pipe, target_names=["Unhappy", "Happy"])
    + " Support Vector Classifier with standard scalar, balanced"
)

That didn't help. SVM with the standard scaler resulted in worst results than SVM just with class_weight=balanced likely because our features are already on the same 1-5 scale.

Let's try something else. Using GridSearch, we might be able to find an optimal set of parameters to maximize performance. Let's also try to see what we get when we optimize for accuracy.

<br>

**Note**: To avoid data leakage (tuning and evaluating on the same dataset), we use nested cross-validation. The inner loop (GridSearchCV) tunes hyperparameters while the outer loop gives an unbiased performance estimate.


In [ ]:
params_grid = {
    "C": [
        0.01,
        0.1,
        1,
        10,
        100,
    ],  # Regularization parameter, low C = more regularization, allows more smoother, general decision boundary
    "kernel": ["linear", "rbf", "poly"],
}

recall_scorer = make_scorer(recall_score, pos_label=0)

In [ ]:
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_recall = GridSearchCV(
    SVC(class_weight="balanced"),
    params_grid,
    scoring=recall_scorer,
    cv=inner_cv,
)

grid_accuracy = GridSearchCV(
    SVC(class_weight="balanced"),
    params_grid,
    scoring="accuracy",
    cv=inner_cv,
)

nested_recall = cross_val_score(grid_recall, X, y, cv=outer_cv, scoring=recall_scorer)
nested_accuracy_with_recall_tuned = cross_val_score(
    grid_recall, X, y, cv=outer_cv, scoring="accuracy"
)

nested_recall_with_accuracy_tuned = cross_val_score(
    grid_accuracy, X, y, cv=outer_cv, scoring=recall_scorer
)
nested_accuracy = cross_val_score(grid_accuracy, X, y, cv=outer_cv, scoring="accuracy")

grid_recall.fit(X, y)

In [ ]:
print(
    "Recall-tuned model: \n"
    + f" Recall: {nested_recall.mean():.3f}, Accuracy: {nested_accuracy_with_recall_tuned.mean():.3f} \n\n"
    + "Accuracy-tuned model: \n"
    + f" Recall: {nested_recall_with_accuracy_tuned.mean():.3f}, Accuracy: {nested_accuracy.mean():.3f} \n\n"
    + f"Inner CV Recall (class 0): {grid_recall.best_score_:.3f} \n"
    + f"Best parameters: {grid_recall.best_params_}"
)

The inner loop found that C=0.01 and a polynomial kernel were the best parameters.

Both recall tuned and accuracy tuned models achieved ~0.57 accuracy (higher than our previous scores) but the recall tuned model had a much higher recall at 0.75.

<br>

**Note**: The nested CV score (~0.75) is different from the fixed parameter inner CV score (~0.67) because nestedCV retunes hyperparameters on each outer fold.

<br>

This shows that recall optimization is the superior solution. We gained performance in recall at no accuracy cost.

This 0.75 recall is the best estimate of how the model will perform on new unseen data.

<br>

For completion, let's verify that 0.57 accuracy is the best we can get with the four models.


In [ ]:
param_grids = {
    "Logistic Regression": {
        "model": LogisticRegression(class_weight="balanced", max_iter=1000),
        "params": {
            "C": [0.01, 0.1, 1, 10, 100],
            "solver": ["lbfgs", "liblinear"],
        },
    },
    "Random Forest": {
        "model": RandomForestClassifier(class_weight="balanced", random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "max_depth": [3, 5, 10, None],
            "min_samples_split": [2, 5, 10],
        },
    },
    "Gradient Boosting": {
        "model": GradientBoostingClassifier(random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "max_depth": [2, 3, 5],
            "learning_rate": [0.01, 0.1, 0.2],
        },
    },
    "SVM": {
        "model": SVC(class_weight="balanced"),
        "params": {
            "C": [0.01, 0.1, 1, 10, 100],
            "kernel": ["linear", "rbf", "poly"],
        },
    },
}

results = []

for name, config in param_grids.items():
    grid = GridSearchCV(
        config["model"],
        config["params"],
        scoring="accuracy",
        cv=inner_cv,
        n_jobs=-1,
    )

    nested_scores = cross_val_score(grid, X, y, cv=outer_cv, scoring="accuracy")

    grid.fit(X, y)

    results.append(
        {
            "Model": name,
            "Accuracy": nested_scores.mean(),
            "Std": nested_scores.std(),
            "Best Params": grid.best_params_,
        }
    )

    print(f"{name}:")
    print(f"  Accuracy: {nested_scores.mean():.3f} ± {nested_scores.std():.3f}")
    print(f"  Best params: {grid.best_params_}\n")

The best result was Gradient Boosting at ~0.63 accuracy with all four tuned models being in the ~0.57 to ~0.63 range.

This suggests that despite tuning across multiple models, we will not be able to achieve a 73% accuracy score and that there is a limitation in the dataset (e.g. small sample size, weak feature to target correlation etc.).

Therefore further optimizing for accuracy does not make sense and as previously stated, recall optimization is the way to go.


### Permutation Feature Importance


Now let's answer the bonus question: which features actually matter for predicting unhappiness? And can we remove any unnecessary ones?

By identifying the minimal feature set on our best model (tuned SVM), we can simplify future surveys while keeping the predictive power.


In [ ]:
tuned_svm = SVC(C=0.01, kernel="poly", class_weight="balanced")
tuned_svm.fit(X, y)

perm = permutation_importance(
    tuned_svm, X, y, scoring=recall_scorer, n_repeats=30, random_state=42
)

importance_df = pd.DataFrame(
    {"Feature": X.columns, "Importance": perm.importances_mean}
).sort_values("Importance", ascending=False)

importance_df


We see that X1 contributed the most followed by X2, X6, X3, X4 then X5 but importance does not equate to necessity. Can we figure out which ones are really necessary?

<br>

**Note**: Recall from the EDA section that X5 had the largest mean difference between unhappy and happy customers. In the permutation importance results, we see that X5 is ranked the least important. This makes sense since X5 correlates with X1 and X3 (see correlation matrix above), and is deemed 'redundant'.


### Feature Engineering - Minimal Feature Set


Recursive feature elimination with cross-validation (RFECV) on SVM requires us to use a kernel other than 'poly'. This does not give us the result we want so we have to take another approach.

Let's try dropping each feature and seeing how recall changes. We first use cross validation with the optimal fixed hyperparameters to find the minimal set.


In [ ]:
all_features = ["X1", "X2", "X3", "X4", "X5", "X6"]

scores = cross_val_score(
    tuned_svm, X[all_features], y, cv=outer_cv, scoring=recall_scorer
)

In [ ]:
print("Baseline (all 6): \n" + f" Recall = {scores.mean():.3f} ± {scores.std():.3f}\n")

print("Drop one feature:")
for drop in all_features:
    features = [f for f in all_features if f != drop]
    scores = cross_val_score(
        tuned_svm, X[features], y, cv=outer_cv, scoring=recall_scorer
    )
    print(f" Drop {drop}: Recall = {scores.mean():.3f} ± {scores.std():.3f}")


We see that

- Dropping X1 hurts our score (=0.58) while dropping X2 increases the score (=~0.69)
- Dropping X4 and X5 does not affect the model much (=0.65)
- Dropping X3 and X6 results in a lower score (=0.63), maybe a bit too low for our liking.

Let's test combinations that drop X2.


In [ ]:
test_sets = {
    "Drop X2": ["X1", "X3", "X4", "X5", "X6"],
    "Drop X2, X4": ["X1", "X3", "X5", "X6"],
    "Drop X2, X5": ["X1", "X3", "X4", "X6"],
    "Drop X2, X4, X5": ["X1", "X3", "X6"],
}

In [ ]:
print("Promising combinations:")
for name, features in test_sets.items():
    scores = cross_val_score(
        tuned_svm, X[features], y, cv=outer_cv, scoring=recall_scorer
    )
    print(f"{name}: Recall = {scores.mean():.3f} ± {scores.std():.3f}")


Dropping X2, X4 and X5 improves recall from ~0.67 to ~0.74.

Let's validate this with the nested CV model (gridsearch). The recall for this model with 6 features was ~0.75.

If we can get ~0.75 with the 3 feature set, then we can confirm that the minimal feature set is [X1, X3, X6].


In [ ]:
nested_reduced = cross_val_score(
    grid_recall, X[["X1", "X3", "X6"]], y, cv=outer_cv, scoring=recall_scorer
)

In [ ]:
print(
    f"Nested CV (3 features): {nested_reduced.mean():.3f} ± {nested_reduced.std():.3f}"
)

The result is now clear. Our minimal feature set is X1, X3, and X6.

We can remove X2, X4 and X5 from future surveys, which reduces survey length by 50% without reducing model performance for identifying unhappy customers.

<br>

**Note**: The high standard deviation is a reflection of the small sample size:

- We used a StratifiedKfold of 5 splits. So 126 samples / 5 = ~25 samples per test fold.
- ~45% are unhappy so 25\*0.45 = ~ 11 unhappy customer per fold
- Identifying one more/less correct = ~9% swing in recall


# Conclusion


**Goal**: Predict customer happiness or unhappiness with 73% accuracy or justify a superior solution.

**Approach**: Focus on identifying unhappy customers (recall for class 0) instead of overall accuracy because this has a higher business value as 45% of customers are dissatisfied.

- Our tuned SVM with nested cross validation (optimal parameters: C=0.01, polynomial kernel) achieved 75.2% recall for unhappy customers. That means we can identify 3 out of 4 unsatisfied customers.

- All four tuned models had an accuracy score in ~57-63% range, which shows that accuracy plateaus at ~63% regardless of optimization but the recall-tuned SVM model also achieved 75% recall vs only 60% for the accuracy tuned SVM model, thereby identifying 15% more unhappy customers at no accuracy cost.

- The top complaints from the customer survey were X2 (contents of my order was as I expected) and X5 (I am satisfied with my courier). It's a universal pain point and adds noise to the model.

- Removing features X2, X4 (I paid a good price for my order) and X5 from the survey resulted in comparable model performance (from 0.752 to 0.753). That is, we reduced survey length by 50% while keeping model performance by going from a 6 feature set to a 3 feature set.

**Recommendations**:

- Prioritize X1, X3, X6 or
- Reduce survey length from 6 questions to 3 questions (X1, X3, X6)
